In [2]:
import sys
import hail as hl
import pandas as pd
import gnomad
from gnomad.resources.grch38 import gnomad
#from gnomad.resources.grch38 import gnomad
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

In [3]:
print("Load pre-filtered BGE data before annotating VEP")
mt = hl.read_matrix_table('gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/01_Var_PreFilterQC.SCZ_BGE.mt')
mt.count()

Initializing Hail with default parameters...


Load pre-filtered BGE data before annotating VEP


/opt/conda/miniconda3/lib/python3.11/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 33155
SPARKMONITOR_LISTENER: Application Started: application_1752605443147_0002 ...Start Time: 1752607773939


Running on Apache Spark version 3.5.0
SparkUI available at http://ychen-cnv-m.us-central1-a.c.pbn-analysis.internal:43201
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.135-034ef3e08116
LOGGING: writing to /home/hail/hail-20250715-1929-0.2.135-034ef3e08116.log


(155072, 70)

In [5]:
print("saving vcf for chrX all - only non-PAR and not pruned...")
mt_chrX = mt.filter_rows(mt.locus.contig == 'chrX') # only chrX
print(mt_chrX.count_rows())
mt_chrX = mt_chrX.filter_rows(mt_chrX.locus.in_x_nonpar()) # only non-PAR for chrX
print(mt_chrX.count_rows()) 

saving vcf for chrX all - only non-PAR and not pruned...


2025-07-15 19:41:26.350 Hail: INFO: wrote table with 155072 rows in 24 partitions to /tmp/persist_TableoUhHYtPUOO


2902
2693


In [6]:
mt_chrX = mt_chrX.drop('info')

In [8]:
print("saving after reducing size...")
hl.export_vcf(mt_chrX, 'gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/sex_check/chrX_NonPAR_only_20250715.vcf.bgz') #

saving after reducing size...


2025-07-15 19:41:59.709 Hail: WARN: export_vcf: ignored the following fields:
    'a_index' (row)
    'was_split' (row)
2025-07-15 19:42:00.957 Hail: WARN: export_vcf found no row field 'info'. Emitting no INFO fields.
2025-07-15 19:42:02.995 Hail: INFO: merging 2 files totalling 212.4K...+ 1) / 1]
2025-07-15 19:42:03.269 Hail: INFO: while writing:
    gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/sex_check/chrX_NonPAR_only_20250715.vcf.bgz
  merge time: 273.704ms


In [9]:
mt_chrX = hl.variant_qc(mt_chrX)
print("AF filtering...")
mt_chrX = mt_chrX.filter_rows(((mt_chrX.variant_qc.AF[0] > 0.001) & (mt_chrX.variant_qc.AF[1] > 0.001)) & ((mt_chrX.variant_qc.AF[0] < 0.999) & (mt_chrX.variant_qc.AF[1] < 0.999)))
print(mt_chrX.count_rows()) # 3039

AF filtering...
2685


In [10]:
print("saving after reducing size...")
hl.export_vcf(mt_chrX, 'gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/sex_check/chrX_NonPAR_only.maf0.001_20250715.vcf.bgz') #

saving after reducing size...


2025-07-15 19:42:55.688 Hail: WARN: export_vcf: ignored the following fields:
    'a_index' (row)
    'was_split' (row)
    'variant_qc' (row)
2025-07-15 19:42:56.967 Hail: WARN: export_vcf found no row field 'info'. Emitting no INFO fields.
2025-07-15 19:42:58.359 Hail: INFO: merging 2 files totalling 210.3K...+ 1) / 1]
2025-07-15 19:42:58.650 Hail: INFO: while writing:
    gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/sex_check/chrX_NonPAR_only.maf0.001_20250715.vcf.bgz
  merge time: 290.186ms


In [12]:
# Let's impute sex now
# Use only sites passing VQSR
mt_temp = mt.filter_rows(mt.filters == hl.empty_set(hl.tstr), keep = True)

# Calls need to be unphased
mt_temp = mt_temp.select_entries(GT = hl.unphased_diploid_gt_index_call(mt_temp.GT.n_alt_alleles()))

# Actually impute sex
imputed_sex = hl.impute_sex(mt_temp.GT)

# Look at results
imputed_sex.show()
# Export results
imputed_sex.drop('is_female').export('gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/sex_check/imputed_sex_bge_20250715.tsv')

,,,,,
s,is_female,f_stat,n_called,expected_homs,observed_homs
str,bool,float64,int64,float64,int64
"""MS111091""",False,9.32e-01,2609,2.31e+03,2589
"""MS137267""",False,9.45e-01,2594,2.30e+03,2578
"""MS163366""",NA,3.61e-01,2623,2.33e+03,2436
"""MS237996""",False,9.39e-01,2606,2.31e+03,2588
"""MS247980""",False,9.72e-01,2609,2.32e+03,2601
"""MS288545""",False,9.63e-01,2619,2.32e+03,2608
"""MS309625""",True,-1.71e-01,2636,2.34e+03,2290
"""MS385647""",False,9.52e-01,2616,2.32e+03,2602


2025-07-15 19:44:14.456 Hail: INFO: Coerced sorted dataset
2025-07-15 19:44:15.592 Hail: INFO: merging 5 files totalling 2.9K...2 + 2) / 4]
2025-07-15 19:44:15.828 Hail: INFO: while writing:
    gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/sex_check/imputed_sex_bge_20250715.tsv
  merge time: 235.098ms
